In [1]:
import potential_fitting
from potential_fitting.database import Database

In [2]:
with Database() as db:
    db.annihilate("confirm")

In [3]:
settings = "H2Omonomer.ini"
unopt_geo = "H2O.xyz"
opt_geo = "H2O.opt.xyz"
normal_modes = "normal_modes.dat"
train_configs = "train_configs.xyz"
test_configs = "test_configs.xyz"
training_set = "training_set.xyz"
client_name = "test_cleint"
test_set = "test_set.xyz"
poly_in = "poly.in"
molecule_in = "A2B1"
poly_directory = "poly"
config = "config.ini"
fit_directory = "fit"
fitted_code = "fitted"

method = "HF"
basis = "STO-3G"
cp = False

In [4]:
potential_fitting.optimize_geometry(settings, unopt_geo, opt_geo)

Beginning geometry optimization using psi4 of H2O with HF/STO-3G.
Optimizer: Optimization complete!
Completed geometry optimization.
File ./H2O.opt.xyz already exists, moving existing ./H2O.opt.xyz to ./H2O.opt.xyz.backup-1 to make way for new file.


In [ ]:
potential_fitting.generate_normal_modes(settings, opt_geo, normal_modes)

In [ ]:
potential_fitting.generate_normal_mode_configurations(settings, opt_geo, normal_modes, train_configs, seed = 3)
potential_fitting.generate_normal_mode_configurations(settings, opt_geo, normal_modes, test_configs, seed = 2)

In [4]:
potential_fitting.init_database(settings, train_configs, method, basis, cp, 'train')
potential_fitting.init_database(settings, opt_geo, method, basis, cp, 'train', optimized = True)
potential_fitting.init_database(settings, test_configs, method, basis, cp, 'test')
potential_fitting.init_database(settings, opt_geo, method, basis, cp, 'test', optimized = True)

Initializing database from xyz file train_configs.xyz directory into database.
Initializing of database successful.
Initializing database from xyz file H2O.opt.xyz directory into database.
Initializing of database successful.
Initializing database from xyz file test_configs.xyz directory into database.
Initializing of database successful.
Initializing database from xyz file H2O.opt.xyz directory into database.
Initializing of database successful.


In [5]:
potential_fitting.database.make_all_jobs(settings, client_name, "jobs", 5)

In [6]:
!python jobs/job_1.py
!python jobs/job_2.py
!python jobs/job_3.py
!python jobs/job_4.py
!python jobs/job_5.py

Maximum threads: 16
Running Job
Molcule: H    6.98646100000000e-01   0.00000000000000e+00   5.44828950000000e-01
H   -7.81218890000000e-01   0.00000000000000e+00   6.14080100000000e-01
O    5.20186000000000e-03   0.00000000000000e+00  -7.30226900000000e-02
Method: HF
Basis: STO-3G
Threads 6
Memory: 1GB
Energy: -74.95723166467671
Maximum threads: 16
Running Job
Molcule: H    8.47369120000000e-01   0.00000000000000e+00   5.83004180000000e-01
H   -8.09120150000000e-01   0.00000000000000e+00   5.50926000000000e-01
O   -2.40958000000000e-03   0.00000000000000e+00  -7.14481300000000e-02
Method: HF
Basis: STO-3G
Threads 6
Memory: 1GB
Energy: -74.95679059503556
Maximum threads: 16
Running Job
Molcule: H    6.69872300000000e-01   0.00000000000000e+00   6.67858430000000e-01
H   -6.57448750000000e-01   0.00000000000000e+00   6.57439200000000e-01
O   -7.82650000000000e-04   0.00000000000000e+00  -8.35116600000000e-02
Method: HF
Basis: STO-3G
Threads 6
Memory: 1GB
Energy: -74.95238385163364
Maximum

In [5]:
potential_fitting.database.read_all_jobs(".")

./job_1


XYZFormatError: The following error occured in the Potential Fitting Library: Invalid xyz formatting on or near line: 'H    6.98646100000000e-01   0.00000000000000e+00   5.44828950000000e-01'. Line format should be line format should be 'charge spin_multiplicity', make sure you are passing in the output of psi4.molecule.save_string_xyz()

In [ ]:
potential_fitting.fill_database(settings, client_name)

In [ ]:

potential_fitting.generate_1b_training_set(settings, training_set, "H2O", method, basis, cp, 'train')
potential_fitting.generate_1b_training_set(settings, test_set, "H2O", method, basis, cp, 'test')


In [ ]:
potential_fitting.generate_poly_input_from_database(settings, database, "H2O", poly_in)

In [ ]:
potential_fitting.generate_polynomials(settings, poly_in, 3, poly_directory)

In [ ]:
potential_fitting.execute_maple(settings, poly_directory)

In [ ]:
potential_fitting.generate_fit_config(settings, molecule_in, config, opt_geo)

In [ ]:
potential_fitting.generate_1b_fit_code(settings, config, molecule_in, poly_in, poly_directory, 3, fit_directory)

In [ ]:
potential_fitting.compile_fit_code(settings, fit_directory)

In [ ]:
potential_fitting.fit_1b_training_set(settings, fit_directory + "/fit-1b", training_set, fit_directory, "fit.nc")

In [ ]:
potential_fitting.fitting.fit_visualizer.make_1b_graphs(fit_directory + "/eval-1b", "fit.nc", database, "H2O", "%", "%", "%", "test")